(poptour_vignette)=

# Properties of populations

This vignette is "getting ahead" of ourselves a bit.
In order for it to be useful, it is best to have a simulated population in hand.
The following hidden code block gives us one.
Feel free to expand it to take a look--the details will make sense once you have studied the later vignettes on setting up and running simulations.
The simulation done here is taken from {ref}`another vignette <gss_vignette>`.

In [1]:
import fwdpy11

pop = fwdpy11.DiploidPopulation(500, 1.0)

rng = fwdpy11.GSLrng(54321)

GSSmo = fwdpy11.GSSmo(
    [
        fwdpy11.Optimum(when=0, optimum=0.0, VS=1.0),
        fwdpy11.Optimum(when=10 * pop.N, optimum=1.0, VS=1.0),
    ]
)

rho = 1000.

p = {
    "nregions": [],
    "gvalue": fwdpy11.Additive(2.0, GSSmo),
    "sregions": [fwdpy11.GaussianS(0, 1., 1, 0.1)],
    "recregions": [fwdpy11.PoissonInterval(0, 1., rho / float(4 * pop.N))],
    "rates": (0.0, 1e-3, None),
    "prune_selected": False,
    "demography": fwdpy11.DiscreteDemography(),
    "simlen": 10 * pop.N + 200,
}
params = fwdpy11.ModelParams(**p)

fwdpy11.evolvets(rng, pop, params, 100, suppress_table_indexing=True)

## Basic properties

In [2]:
print(f"Total number of diploids = {pop.N}")
print(f"Current generation = {pop.generation}")

Total number of diploids = 500
Current generation = 5200


## Mutations

Mutations are instances of {class}`fwdpy11.Mutation` stored in {attr}`fwdpy11.DiploidPopulation.mutations`.
These objects do not have nice representations in Python:

In [3]:
[i for i in pop.mutations[:5]]

So, let's just look at the fields of the first mutation:

In [4]:
m = pop.mutations[0]
print(f"position = {m.pos}\neffect_size = {m.s}\ndominance = {m.h}\norigin time = {m.g}")

position = 0.4431572693865746
effect_size = 0.10946401078096415
dominance = 1.0
origin time = 5020


{class}`fwdpy11.Mutation` has other attributes that are not relevant to the type of simulation done here.

## Diploids

Diploids are instances of {class}`fwdpy11.DiploidGenotype`.
They are stored in `fwdpy11.DiploidPopulation.diploids`.
Diploids store the indexes of their individual genomes, which we describe below.

In [5]:
[i for i in pop.diploids[:5]]

[DiploidGenotype(first=562, second=41),
 DiploidGenotype(first=338, second=0),
 DiploidGenotype(first=550, second=289),
 DiploidGenotype(first=633, second=2),
 DiploidGenotype(first=4, second=5)]

## Diploid meta data

Diploid individuals have associated data, stored in {attr}`fwdpy11.DiploidPopulation.diploid_metadata`.
The meta data are instances of {class}`fwdpy11.DiploidMetadata`.
That class is also a {class}`numpy.dtype`, allowing us to access the raw data efficiently as a {class}`numpy.recarray`:

In [6]:
import numpy as np
md = np.array(pop.diploid_metadata, copy=False)
md[:5]

array([(0.94224157, 0., 0.99833337, [0., 0., 0.], 0, [354, 299], 0, 0, [0, 1]),
       (0.84475778, 0., 0.98802224, [0., 0., 0.], 1, [ 87, 322], 0, 0, [2, 3]),
       (0.84475778, 0., 0.98802224, [0., 0., 0.], 2, [349, 242], 0, 0, [4, 5]),
       (0.84475778, 0., 0.98802224, [0., 0., 0.], 3, [403, 428], 0, 0, [6, 7]),
       (0.8368247 , 0., 0.98677514, [0., 0., 0.], 4, [289, 428], 0, 0, [8, 9])],
      dtype=[('g', '<f8'), ('e', '<f8'), ('w', '<f8'), ('geography', '<f8', (3,)), ('label', '<u8'), ('parents', '<u8', (2,)), ('deme', '<i4'), ('sex', '<i4'), ('nodes', '<i4', (2,))])

The field names of the record array exactly match the attribute names of {class}`fwdpy11.DiploidMetadata`.

The record arrays allow efficient calculation of important quantities:

In [7]:
print(f"Mean trait value = {md['g'].mean():0.4f}.\nMean fitness = {md['w'].mean():0.4f}")

Mean trait value = 0.8766.
Mean fitness = 0.9914


### Ancient samples

When ancient/preserved samples are recorded during simulations, their meta data are stored in :attr:`fwdpy11.DiploidPopulation.ancient_sample_metadata`.

## Haploid genomes

Haploid genomes are instances of {class}`fwdpy11.HaploidGenome`.
These objects contain indexes to mutations.
Let's look at the effect sizes and origin times of mutations in the first non-empty genome:

In [8]:
def mut_info(pop, i):
    rv = False
    for m in pop.haploid_genomes[i].smutations:
        print(f"effect size = {pop.mutations[m].s:0.4f}, origin time = {pop.mutations[m].g}")
        rv = True
    return rv
    
for i in pop.diploids:
    if mut_info(pop, i.first) is True:
        break
    elif mut_info(pop, i.second) is True:
        break

effect size = -0.0194, origin time = 1252
effect size = 0.0079, origin time = 4582
effect size = 0.0546, origin time = 141
effect size = 0.1095, origin time = 5020
effect size = 0.1377, origin time = 4998
effect size = 0.1383, origin time = 5000
effect size = 0.0159, origin time = 4396
effect size = -0.0177, origin time = 1052
effect size = 0.0115, origin time = 1539


### Some details

Neutral mutations are never added to haploid genomes.

## Tables

A population contains an instance of {class}`fwdpy11.TableCollection` which is used to represent the genetic ancestry of the sample using the methods described in {cite}`Kelleher2018-mc`.

The table contents are described in the class documentation.
All tables can be accessed either as Python objects or as {class}`numpy.recarray` objects as we say above for diploid meta data.